# Imports, install and mount

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
my_local_drive='/content/gdrive/MyDrive/SL2024/datas'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd
print("DRIVE MOUNTED")

Mounted at /content/gdrive
/content/gdrive/MyDrive/SL2024/datas
DRIVE MOUNTED


In [34]:
! pip install cuda
! pip install torch_geometric
! pip install dgl

import torch
import torch.optim as optim
import os.path as osp
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
import pandas as pd
from tqdm import tqdm
from torch_geometric.loader import DataLoader
import dgl
import os


ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu' # Tip : Use cpu for debugging
print(device)

cpu


# TSV to pyg.Data

We want :
- x (tensorised and processed node attributes) (1 for now)
- edge_index (a tensor of shape (num_edges, 2) indicates the source node index and the destination node index)
- edge_attr (a tensor that indicate the type of each edge)
- y (edge labels - optional, can be defined as node labels if needed)

## Reading and mapping graph

What we want : Create a Data object with all the properties I want to use later

	- x (tensorised and processed node attributes) (Not for now)
	- edge_index (a tensor of shape (num_edges, 2) indicates the source node index and the destination node index)
	- y (desired edge labels - optional, can be defined as node labels if needed) (Not for now)
	- any other things you want to use later

In [6]:
file_path = "/content/gdrive/MyDrive/SL2024/datas/little_iric.tsv"
iric = pd.read_csv(file_path, delimiter='\t')
display(iric)

,subject,predicate,object
0,GO:0000001,is_a,GO:0048311
1,GO:0000001,is_a,GO:0048308
2,GO:0000002,is_a,GO:0007005
3,GO:0000003,is_a,GO:0008150
4,GO:0000006,is_a,GO:0005385
...,...,...,...
9994,GO:0009920,is_a,GO:0000919
9995,GO:0009920,is_a,GO:1903047
9996,GO:0009921,is_a,GO:0098797
9997,GO:0009922,is_a,GO:0016747


In [7]:
# Mapping entities to ids

entity_set = set(iric['object']).union(set(iric['subject']))
entity_to_index = {entity: i for i, entity in enumerate(entity_set)}
relation_set = set(iric['predicate'])
relation_to_index = {relation: i for i, relation in enumerate(relation_set)}

mapped_iric = iric.copy()
mapped_iric['object'] = mapped_iric['object'].apply(lambda x: entity_to_index[x])
mapped_iric['subject'] = mapped_iric['subject'].apply(lambda x: entity_to_index[x])
mapped_iric['predicate'] = mapped_iric['predicate'].apply(lambda x: relation_to_index[x])

display(mapped_iric)
# IMPORTANT : check that mappings of subjects start at 0.

print("Minima in mappings :")
print('subject :', min(mapped_iric['subject']))
print('predicate :',min(mapped_iric['predicate']))
print('object :',min(mapped_iric['object']))

print(entity_to_index)
print(relation_to_index)

index_to_entity = {v: k for k, v in entity_to_index.items()}
index_to_relation = {v: k for k, v in relation_to_index.items()}
print(index_to_entity)
print(index_to_relation)

,subject,predicate,object
0,3557,0,7816
1,3557,0,7333
2,134,0,7800
3,5866,0,6087
4,8550,0,6251
...,...,...,...
9994,6146,0,3600
9995,6146,0,2736
9996,7936,0,8316
9997,59,0,4088


Minima in mappings :
subject : 4
predicate : 0
object : 0
{'GO:0046457': 0, 'GO:0048531': 1, 'GO:0048232': 2, 'GO:0045637': 3, 'GO:0000297': 4, 'GO:0006378': 5, 'GO:0008424': 6, 'GO:0009566': 7, 'GO:0000242': 8, 'GO:0006610': 9, 'GO:0000406': 10, 'GO:0006111': 11, 'GO:0009059': 12, 'GO:0003839': 13, 'GO:0002289': 14, 'GO:0000145': 15, 'GO:0006749': 16, 'GO:0001810': 17, 'GO:0006911': 18, 'GO:0046068': 19, 'GO:0009331': 20, 'GO:0016892': 21, 'GO:0046152': 22, 'GO:0003003': 23, 'GO:0003919': 24, 'GO:0004740': 25, 'GO:0009381': 26, 'GO:0005754': 27, 'GO:0000817': 28, 'GO:0006573': 29, 'GO:0005794': 30, 'GO:0009146': 31, 'GO:0009669': 32, 'GO:0004336': 33, 'GO:0001691': 34, 'GO:0014075': 35, 'GO:0001798': 36, 'GO:0003084': 37, 'GO:0006207': 38, 'GO:0008757': 39, 'GO:0051239': 40, 'GO:0009639': 41, 'GO:0140101': 42, 'GO:0006999': 43, 'GO:0019903': 44, 'GO:0016812': 45, 'GO:0009192': 46, 'GO:0002254': 47, 'GO:0045254': 48, 'GO:0002732': 49, 'GO:0001716': 50, 'GO:0004551': 51, 'GO:0003370': 5

## Building init vars for Data:

In [8]:
# # Initial nodes states :
# x = torch.ones(len(entity_set), 1)  # Chaque nœud a 1 pour état initial
# print('X : \n',x)

# Edges index
heads = list(mapped_iric['subject'])
tails = list(mapped_iric['object'])
edge_index = torch.tensor([heads,tails], dtype=torch.long)
print('\nEDGE INDEX : \n',edge_index)

# edges states
edge_attributes = torch.tensor(mapped_iric['predicate'])
print('\nEDGES ATTRIBUTES : \n',edge_attributes)

iric_pyg = Data(
                # x = x,
                num_nodes = len(entity_set),
                edge_index = edge_index,
                edge_attr = edge_attributes)
print('\nDATASET :\n',iric_pyg)
iric_pyg.edge_index


print("\nDataset looks valid ? \n",iric_pyg.validate(raise_on_error=True))


EDGE INDEX : 
 tensor([[3557, 3557,  134,  ..., 7936,   59, 1172],
        [7816, 7333, 7800,  ..., 8316, 4088, 6273]])

EDGES ATTRIBUTES : 
 tensor([0, 0, 0,  ..., 0, 0, 0])

DATASET :
 Data(edge_index=[2, 9999], edge_attr=[9999], num_nodes=8859)

Dataset looks valid ? 
 True


# Setting up datas and model


## Splitting dataset

In [9]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(
                            num_val = 0.1,
                            num_test = 0.1,
                            is_undirected=False,
                            add_negative_train_samples=False,
                            )

In [10]:
train_data, val_data, test_data = transform(iric_pyg)

print(f"Hole Dataset :\n {iric_pyg}\n\nTrain:\n{train_data}\n\nTest :\n{test_data}\n\nValidation :\n{val_data}")
# Il ne faut pas regarder num_edges parce que RLS cache les arêtes mais ne les sort pas du graph.
# print(f"Number of edges in datasets : \n  Hole Dataset : {iric_pyg.num_edges}\n\n  Train: {train_data.num_edges}\n\n  Test : {test_data.num_edges}\n\n  Validation : {val_data.num_edges}")
print(f"Number of edges in datasets : \n  Train: {list(train_data.edge_label.size())[0]}\n\n  Test : {list(test_data.edge_label.size())[0]}\n\n  Validation : {list(val_data.edge_label.size())[0]}")

Hole Dataset :
 Data(edge_index=[2, 9999], edge_attr=[9999], num_nodes=8859)

Train:
Data(edge_index=[2, 8001], edge_attr=[8001], num_nodes=8859, edge_label=[8001], edge_label_index=[2, 8001])

Test :
Data(edge_index=[2, 9000], edge_attr=[9000], num_nodes=8859, edge_label=[1998], edge_label_index=[2, 1998])

Validation :
Data(edge_index=[2, 8001], edge_attr=[8001], num_nodes=8859, edge_label=[1998], edge_label_index=[2, 1998])
Number of edges in datasets : 
  Train: 8001

  Test : 1998

  Validation : 1998


In [11]:
# # Non pertinent

# train_edges = set([(train.edge_index[0][i].item(), train.edge_index[1][i].item()) for i in range(min(train.edge_index.size(1), train.edge_index.size(1)))])
# test_edges = set([(test.edge_index[0][i].item(), test.edge_index[1][i].item()) for i in range(min(test.edge_index.size(1), test.edge_index.size(1)))])
# val_edges = set([(val.edge_index[0][i].item(), val.edge_index[1][i].item()) for i in range(min(val.edge_index.size(1), val.edge_index.size(1)))])
# print(len(test_edges.intersection(val_edges).intersection(train_edges)))
# print(test_edges)
# mapped_iric.loc[mapped_iric['subject'] == 2183]
# mapped_iric.loc[mapped_iric['subject'] == 928]

## Iniating model

In [12]:
print(train_data.num_nodes)
print(train_data.num_edge_types)
print(train_data.__dict__)
print(train_data.edge_index[0].size())
print(train_data.edge_index[1].size())
print(train_data.edge_attr.size())
print(train_data.edge_attr)
print(train_data.num_nodes)

8859
1
{'_edge_attr_cls': <class 'torch_geometric.data.data.DataEdgeAttr'>, '_tensor_attr_cls': <class 'torch_geometric.data.data.DataTensorAttr'>, '_store': {'edge_index': tensor([[6671, 5348, 4406,  ..., 7868, 6037, 2593],
        [ 657, 4867, 7426,  ..., 7484, 8470, 3233]]), 'edge_attr': tensor([0, 0, 0,  ..., 0, 0, 0]), 'num_nodes': 8859, 'edge_label': tensor([1., 1., 1.,  ..., 1., 1., 1.]), 'edge_label_index': tensor([[6671, 5348, 4406,  ..., 7868, 6037, 2593],
        [ 657, 4867, 7426,  ..., 7484, 8470, 3233]])}}
torch.Size([8001])
torch.Size([8001])
torch.Size([8001])
tensor([0, 0, 0,  ..., 0, 0, 0])
8859


In [22]:
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

model = ComplEx(
    num_nodes=train_data.num_nodes,
    # num_relations=train_data.num_edge_types,
    num_relations = len(train_data.edge_index.size()),
    hidden_channels=176,
).to(device)

head_index = train_data.edge_index[0]
tail_index = train_data.edge_index[1]
rel_type = train_data.edge_attr

loader = model.loader(
    head_index = head_index,
    tail_index = tail_index,
    rel_type = rel_type,
    batch_size=4096,
    shuffle=True,
)

optimizer = optim.Adam(model.parameters(),
                      #  lr=0.01, weight_decay=1e-6
                       )

@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=test_data.edge_index[0],
        tail_index=test_data.edge_index[1],
        rel_type=test_data.edge_attr,
        batch_size=1000,
        k=10,
    )

def train():
    model.train()
    total_loss = total_examples = 0

    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


for epoch in range(1, 1000):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 200 == 0:
        rank, mrr, hits = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}',
              f'Val MRR: {mrr:.4f}, Val Hits@10: {hits:.4f}')

rank, mrr, hits_at_10 = test(test_data)
print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, '
      f'Test Hits@10: {hits_at_10:.4f}')

Epoch: 001, Loss: 0.6931
Epoch: 002, Loss: 0.6929
Epoch: 003, Loss: 0.6927
Epoch: 004, Loss: 0.6925
Epoch: 005, Loss: 0.6922
Epoch: 006, Loss: 0.6920
Epoch: 007, Loss: 0.6917
Epoch: 008, Loss: 0.6914
Epoch: 009, Loss: 0.6911
Epoch: 010, Loss: 0.6907
Epoch: 011, Loss: 0.6903
Epoch: 012, Loss: 0.6898
Epoch: 013, Loss: 0.6893
Epoch: 014, Loss: 0.6887
Epoch: 015, Loss: 0.6881
Epoch: 016, Loss: 0.6874
Epoch: 017, Loss: 0.6867
Epoch: 018, Loss: 0.6858
Epoch: 019, Loss: 0.6848
Epoch: 020, Loss: 0.6838
Epoch: 021, Loss: 0.6827
Epoch: 022, Loss: 0.6814
Epoch: 023, Loss: 0.6800
Epoch: 024, Loss: 0.6785
Epoch: 025, Loss: 0.6769
Epoch: 026, Loss: 0.6750
Epoch: 027, Loss: 0.6731
Epoch: 028, Loss: 0.6710
Epoch: 029, Loss: 0.6687
Epoch: 030, Loss: 0.6663
Epoch: 031, Loss: 0.6636
Epoch: 032, Loss: 0.6608
Epoch: 033, Loss: 0.6578
Epoch: 034, Loss: 0.6545
Epoch: 035, Loss: 0.6512
Epoch: 036, Loss: 0.6475
Epoch: 037, Loss: 0.6437
Epoch: 038, Loss: 0.6397
Epoch: 039, Loss: 0.6355
Epoch: 040, Loss: 0.6310


100%|██████████| 9000/9000 [03:16<00:00, 45.78it/s]


Epoch: 200, Val Mean Rank: 363.49 Val MRR: 0.7624, Val Hits@10: 0.9101
Epoch: 201, Loss: 0.3573
Epoch: 202, Loss: 0.3573
Epoch: 203, Loss: 0.3550
Epoch: 204, Loss: 0.3565
Epoch: 205, Loss: 0.3567
Epoch: 206, Loss: 0.3560
Epoch: 207, Loss: 0.3567
Epoch: 208, Loss: 0.3554
Epoch: 209, Loss: 0.3567
Epoch: 210, Loss: 0.3552
Epoch: 211, Loss: 0.3560
Epoch: 212, Loss: 0.3571
Epoch: 213, Loss: 0.3563
Epoch: 214, Loss: 0.3554
Epoch: 215, Loss: 0.3554
Epoch: 216, Loss: 0.3557
Epoch: 217, Loss: 0.3551
Epoch: 218, Loss: 0.3565
Epoch: 219, Loss: 0.3544
Epoch: 220, Loss: 0.3548
Epoch: 221, Loss: 0.3550
Epoch: 222, Loss: 0.3540
Epoch: 223, Loss: 0.3541
Epoch: 224, Loss: 0.3551
Epoch: 225, Loss: 0.3544
Epoch: 226, Loss: 0.3534
Epoch: 227, Loss: 0.3529
Epoch: 228, Loss: 0.3535
Epoch: 229, Loss: 0.3537
Epoch: 230, Loss: 0.3545
Epoch: 231, Loss: 0.3548
Epoch: 232, Loss: 0.3551
Epoch: 233, Loss: 0.3533
Epoch: 234, Loss: 0.3526
Epoch: 235, Loss: 0.3538
Epoch: 236, Loss: 0.3531
Epoch: 237, Loss: 0.3533
Epoc

100%|██████████| 9000/9000 [03:20<00:00, 44.87it/s]


Epoch: 400, Val Mean Rank: 515.10 Val MRR: 0.7601, Val Hits@10: 0.9057
Epoch: 401, Loss: 0.1205
Epoch: 402, Loss: 0.1193
Epoch: 403, Loss: 0.1138
Epoch: 404, Loss: 0.1138
Epoch: 405, Loss: 0.1102
Epoch: 406, Loss: 0.1091
Epoch: 407, Loss: 0.1078
Epoch: 408, Loss: 0.1052
Epoch: 409, Loss: 0.1043
Epoch: 410, Loss: 0.1018
Epoch: 411, Loss: 0.1006
Epoch: 412, Loss: 0.0968
Epoch: 413, Loss: 0.0969
Epoch: 414, Loss: 0.0952
Epoch: 415, Loss: 0.0914
Epoch: 416, Loss: 0.0913
Epoch: 417, Loss: 0.0899
Epoch: 418, Loss: 0.0875
Epoch: 419, Loss: 0.0866
Epoch: 420, Loss: 0.0846
Epoch: 421, Loss: 0.0831
Epoch: 422, Loss: 0.0829
Epoch: 423, Loss: 0.0804
Epoch: 424, Loss: 0.0783
Epoch: 425, Loss: 0.0786
Epoch: 426, Loss: 0.0773
Epoch: 427, Loss: 0.0758
Epoch: 428, Loss: 0.0733
Epoch: 429, Loss: 0.0722
Epoch: 430, Loss: 0.0721
Epoch: 431, Loss: 0.0712
Epoch: 432, Loss: 0.0693
Epoch: 433, Loss: 0.0670
Epoch: 434, Loss: 0.0680
Epoch: 435, Loss: 0.0664
Epoch: 436, Loss: 0.0656
Epoch: 437, Loss: 0.0640
Epoc

...

Epoch: 600, Val Mean Rank: 528.37 Val MRR: 0.7605, Val Hits@10: 0.9057
Epoch: 601, Loss: 0.0138
Epoch: 602, Loss: 0.0131
Epoch: 603, Loss: 0.0125
Epoch: 604, Loss: 0.0127
Epoch: 605, Loss: 0.0125
Epoch: 606, Loss: 0.0134
Epoch: 607, Loss: 0.0126
Epoch: 608, Loss: 0.0115
Epoch: 609, Loss: 0.0135
Epoch: 610, Loss: 0.0125
Epoch: 611, Loss: 0.0131
Epoch: 612, Loss: 0.0118
Epoch: 613, Loss: 0.0117
Epoch: 614, Loss: 0.0110
Epoch: 615, Loss: 0.0112
Epoch: 616, Loss: 0.0118
Epoch: 617, Loss: 0.0120
Epoch: 618, Loss: 0.0106
Epoch: 619, Loss: 0.0118
Epoch: 620, Loss: 0.0121
Epoch: 621, Loss: 0.0115
Epoch: 622, Loss: 0.0123
Epoch: 623, Loss: 0.0112
Epoch: 624, Loss: 0.0115
Epoch: 625, Loss: 0.0108
Epoch: 626, Loss: 0.0123
Epoch: 627, Loss: 0.0104
Epoch: 628, Loss: 0.0111
Epoch: 629, Loss: 0.0123
Epoch: 630, Loss: 0.0108
Epoch: 631, Loss: 0.0116
Epoch: 632, Loss: 0.0106
Epoch: 633, Loss: 0.0119
Epoch: 634, Loss: 0.0099
Epoch: 635, Loss: 0.0097
Epoch: 636, Loss: 0.0104
Epoch: 637, Loss: 0.0096
Epoc

100%|██████████| 9000/9000 [03:11<00:00, 46.94it/s]


Epoch: 800, Val Mean Rank: 524.16 Val MRR: 0.7609, Val Hits@10: 0.9068
Epoch: 801, Loss: 0.0047
Epoch: 802, Loss: 0.0054
Epoch: 803, Loss: 0.0054
Epoch: 804, Loss: 0.0056
Epoch: 805, Loss: 0.0056
Epoch: 806, Loss: 0.0055
Epoch: 807, Loss: 0.0059
Epoch: 808, Loss: 0.0064
Epoch: 809, Loss: 0.0056
Epoch: 810, Loss: 0.0063
Epoch: 811, Loss: 0.0053
Epoch: 812, Loss: 0.0051
Epoch: 813, Loss: 0.0055
Epoch: 814, Loss: 0.0057
Epoch: 815, Loss: 0.0056
Epoch: 816, Loss: 0.0053
Epoch: 817, Loss: 0.0042
Epoch: 818, Loss: 0.0065
Epoch: 819, Loss: 0.0044
Epoch: 820, Loss: 0.0040
Epoch: 821, Loss: 0.0056
Epoch: 822, Loss: 0.0047
Epoch: 823, Loss: 0.0038
Epoch: 824, Loss: 0.0046
Epoch: 825, Loss: 0.0052
Epoch: 826, Loss: 0.0055
Epoch: 827, Loss: 0.0047
Epoch: 828, Loss: 0.0053
Epoch: 829, Loss: 0.0040
Epoch: 830, Loss: 0.0055
Epoch: 831, Loss: 0.0050
Epoch: 832, Loss: 0.0042
Epoch: 833, Loss: 0.0043
Epoch: 834, Loss: 0.0037
Epoch: 835, Loss: 0.0047
Epoch: 836, Loss: 0.0048
Epoch: 837, Loss: 0.0042
Epoc

100%|██████████| 9000/9000 [03:17<00:00, 45.65it/s]

Test Mean Rank: 520.72, Test MRR: 0.7611, Test Hits@10: 0.9073


# Looking for embeddings

In [23]:
copy = model
print(model(torch.tensor([50]),torch.tensor([0]),torch.tensor([50]))==copy(torch.tensor([50]),torch.tensor([0]),torch.tensor([50])))

tensor([True])


In [24]:
model_save_path = "TestComplEx"
loc_path = %pwd
a = model(torch.tensor([50]),torch.tensor([0]),torch.tensor([50]))
torch.save(model.state_dict(), model_save_path)
print(f'Model saved at {loc_path}/{model_save_path}')


Model saved at /content/gdrive/MyDrive/SL2024/datas/TestComplEx


In [25]:
reloaded = ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = len(train_data.edge_index.size()),
    hidden_channels=176,
)
reloaded.load_state_dict(torch.load(model_save_path))
b = reloaded(torch.tensor([50]),torch.tensor([0]),torch.tensor([50]))
print(a==b)

tensor([True])


In [26]:
'''
Attributs pour accéder aux embeddings :

        head_re = self.node_emb(head_index)
        head_im = self.node_emb_im(head_index)
        rel_re = self.rel_emb(rel_type)
        rel_im = self.rel_emb_im(rel_type)
        tail_re = self.node_emb(tail_index)
        tail_im = self.node_emb_im(tail_index)
'''

'\nAttributs pour accéder aux embeddings :\n\n        head_re = self.node_emb(head_index)\n        head_im = self.node_emb_im(head_index)\n        rel_re = self.rel_emb(rel_type)\n        rel_im = self.rel_emb_im(rel_type)\n        tail_re = self.node_emb(tail_index)\n        tail_im = self.node_emb_im(tail_index)\n'

In [33]:
print(model.node_emb_im(torch.tensor(0)))
print(reloaded.node_emb_im(torch.tensor(0)))

tensor([-0.0589,  0.0367, -0.1227, -0.1272,  0.0547,  0.0415, -0.0262, -0.1080,
        -0.0060, -0.0653, -0.1918,  0.0303,  0.0031, -0.2006, -0.0106,  0.0367,
         0.0109, -0.0290, -0.0172,  0.0319, -0.1774,  0.1907,  0.0374, -0.0090,
         0.0403, -0.0032, -0.4255, -0.0044,  0.1516,  0.0141, -0.1626, -0.3987,
         0.0742, -0.1282,  0.0055,  0.1069,  0.0674,  0.0234, -0.1437,  0.1150,
         0.3460,  0.0970, -0.0513,  0.0808, -0.0595, -0.0289, -0.0502, -0.0366,
         0.0292, -0.0326,  0.0126, -0.0932,  0.0133,  0.0206,  0.0598, -0.0762,
         0.0307,  0.0332,  0.3387, -0.0135, -0.0203,  0.0557, -0.0091, -0.0118,
        -0.0296, -0.0289, -0.0036,  0.0046, -0.0657, -0.1211, -0.2824,  0.0254,
        -0.0214, -0.0979, -0.0087, -0.0051,  0.0108, -0.1134,  0.0519, -0.0292,
         0.0134,  0.0432,  0.2955,  0.2161,  0.0079, -0.0362,  0.0286,  0.0147,
        -0.0619,  0.0124,  0.0653, -0.1503,  0.0040, -0.0641,  0.0198,  0.0542,
        -0.0542,  0.1988,  0.0334, -0.07

In [28]:
model.rel_emb(torch.tensor(0))

tensor([-1.3056,  0.8389, -1.3701,  0.8144,  0.8285,  0.7393, -0.9134, -1.7613,
         0.8044, -0.9213, -1.3294,  0.8359,  0.8293, -1.5244, -0.8299,  0.8155,
         0.8297,  0.8189, -1.5264, -1.0597, -1.3411, -1.4177,  0.8089,  0.8689,
        -0.9451,  0.8413, -1.6316,  0.8203, -1.1918,  0.8397, -1.4558, -1.7113,
         0.8217, -1.5251, -0.9698, -1.3193,  0.8824, -0.8533, -1.4562, -1.0628,
        -1.6017, -1.2388, -0.9300, -1.0615,  0.8353,  0.7720,  0.7432,  0.7818,
        -1.0213,  0.8262,  0.7922,  0.8231, -1.0170,  0.8029,  0.7609, -1.2035,
         0.8198,  0.8498, -1.4864, -1.4017,  0.8082, -1.1033,  0.8207, -0.9220,
         0.7991,  0.7825, -1.3517,  0.8168, -1.0172, -1.4119, -1.6693,  0.8586,
         0.8120,  0.8518,  0.8441,  0.7882, -0.9118, -1.2723,  0.8182,  0.8378,
        -1.2971,  0.8156, -1.6043, -1.6675,  0.8478,  0.8559, -0.9960,  0.8466,
         0.7947, -1.1896,  0.8054, -1.0948,  0.7922,  0.8658,  0.8030, -1.1277,
        -1.3471, -1.5790,  0.8608,  0.80